In [1]:
import pylab
import calendar
import numpy as np
import pandas as pd
import seaborn as sn
from scipy import stats
import missingno as msno
from datetime import datetime
import matplotlib.pyplot as plt
import warnings
pd.options.mode.chained_assignment = None
warnings.filterwarnings("ignore", category=DeprecationWarning)

%matplotlib inline

In [2]:
#读取数据
dailyData = pd.read_csv('/Users/ranmo/Desktop/机器学习项目集/华盛顿特区首都自行车租赁预测/data/train.csv')

### 数据总结
作为第一步，让我们对数据集执行三个简单步骤
* 数据集大小
* 通过打印几行数据来看看
* 查看数据的变量类型

In [3]:
dailyData.shape

(10886, 12)

In [4]:
dailyData.head(2)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40


In [5]:
dailyData.dtypes

datetime       object
season          int64
holiday         int64
workingday      int64
weather         int64
temp          float64
atemp         float64
humidity        int64
windspeed     float64
casual          int64
registered      int64
count           int64
dtype: object

### 特征工程
从以上结果可以看出，“season”、“holiday”、“workingday”和“weather”等栏应该是“类别”数据类型，但目前的数据类型是“int”。让我们以以下方式转换数据集，以便我们可以开始使用EDA
* 从“datatime”栏中创建新列“data”、“hour”、“weekDay”、“month”
* 将“season”、“holiday”、“workingday”和“weather”的数据类型强制转换成分类类型
* 删除datetime列，因为我们已经从其中提取了有用的特性


In [6]:
dailyData["data"] = dailyData.datetime.apply(lambda x : x.split()[0])
dailyData["hour"] = dailyData.datetime.apply(lambda x : x.split()[1].split(":")[0])
dailyData["weekday"] = dailyData.data.apply(lambda dataString : calendar.day_name[datetime.strptime(dataString,"%Y-%m-%d").weekday()])
dailyData["month"] = dailyData.data.apply(lambda dataString : calendar.month_name[datetime.strptime(dataString,"%Y-%m-%d").month])
dailyData["season"] = dailyData.season.map({1: "Spring", 2:"Summer", 3: "Fall", 4: "Winter"})
dailyData["weather"] = dailyData.weather.map({1: " Clear + Few clouds + Partly cloudy + Partly cloudy",\
                                        2 : " Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist ", \
                                        3 : " Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds", \
                                        4 :" Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog " })

In [7]:
dailyData.head(2)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,data,hour,weekday,month
0,2011-01-01 00:00:00,Spring,0,0,Clear + Few clouds + Partly cloudy + Partly c...,9.84,14.395,81,0.0,3,13,16,2011-01-01,00,Saturday,January
1,2011-01-01 01:00:00,Spring,0,0,Clear + Few clouds + Partly cloudy + Partly c...,9.02,13.635,80,0.0,8,32,40,2011-01-01,01,Saturday,January


In [8]:
categoryVariableList = ["hour","weekday","month","season","weather","holiday","workingday"]
for var in categoryVariableList:
    dailyData[var] = dailyData[var].astype("category")

In [9]:
dailyData.head(2)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,data,hour,weekday,month
0,2011-01-01 00:00:00,Spring,0,0,Clear + Few clouds + Partly cloudy + Partly c...,9.84,14.395,81,0.0,3,13,16,2011-01-01,00,Saturday,January
1,2011-01-01 01:00:00,Spring,0,0,Clear + Few clouds + Partly cloudy + Partly c...,9.02,13.635,80,0.0,8,32,40,2011-01-01,01,Saturday,January


In [10]:
dailyData.dtypes

datetime        object
season        category
holiday       category
workingday    category
weather       category
temp           float64
atemp          float64
humidity         int64
windspeed      float64
casual           int64
registered       int64
count            int64
data            object
hour          category
weekday       category
month         category
dtype: object

In [11]:
dailyData = dailyData.drop(["datetime"], axis = 1)

### 让我们从变量数据类型的非常简单的可视化开始
处理缺失值

In [12]:
dailyData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 15 columns):
season        10886 non-null category
holiday       10886 non-null category
workingday    10886 non-null category
weather       10886 non-null category
temp          10886 non-null float64
atemp         10886 non-null float64
humidity      10886 non-null int64
windspeed     10886 non-null float64
casual        10886 non-null int64
registered    10886 non-null int64
count         10886 non-null int64
data          10886 non-null object
hour          10886 non-null category
weekday       10886 non-null category
month         10886 non-null category
dtypes: category(7), float64(3), int64(4), object(1)
memory usage: 757.0+ KB


In [13]:
dailyData.isnull().sum()

season        0
holiday       0
workingday    0
weather       0
temp          0
atemp         0
humidity      0
windspeed     0
casual        0
registered    0
count         0
data          0
hour          0
weekday       0
month         0
dtype: int64

### 数据分布分析

In [14]:
msno.matrix(dailyData,figsize=(12,5))

<Figure size 864x360 with 2 Axes>

### 异常值分析

In [15]:
fig, axes = plt.subplots(nrows=2,ncols=2)
fig.set_size_inches(12, 10)
sn.boxplot(data=dailyData,y="count",orient="v",ax=axes[0][0])
sn.boxplot(data=dailyData,y="count",x="season",orient="v",ax=axes[0][1])
sn.boxplot(data=dailyData,y="count",x="hour",orient="v",ax=axes[1][0])
sn.boxplot(data=dailyData,y="count",x="workingday",orient="v",ax=axes[1][1])

axes[0][0].set(ylabel='Count',title="Box Plot On Count")
axes[0][1].set(xlabel='Season', ylabel='Count',title="Box Plot On Count Across Season")
axes[1][0].set(xlabel='Hour Of The Day', ylabel='Count',title="Box Plot On Count Across Hour Of The Day")
axes[1][1].set(xlabel='Working Day', ylabel='Count',title="Box Plot On Count Across Working Day")

[Text(0, 0.5, 'Count'),
 Text(0.5, 0, 'Working Day'),
 Text(0.5, 1.0, 'Box Plot On Count Across Working Day')]

<Figure size 864x720 with 4 Axes>

乍一看，“count”变量包含了许多异常值数据点，这些数据点使分布向右倾斜(因为有更多的数据点超出了外部四分位数的限制)，但除此之外，还可以从下面给出的简单方框图中进行以下推断。
* 春季的数量相对较少，方块中的中值下降就证明了这一点。
* “一天中的每一小时”的盒式情节很有趣，上午7时至上午8时和下午5时至下午6时的中值相对较高。这可归因于当时的正规学校和办公室用户。
* 大多数异常点主要来自“工作日”而不是“非工作日”。从图4可以看到。

### 移除计数列中的异常值。

In [16]:
dailyDataWithoutOutliers = dailyData[np.abs(dailyData["count"]-dailyData["count"].mean())<=(3*dailyData["count"].std())] 

In [17]:
print ("Shape Of The Before Ouliers: ",dailyData.shape)
print ("Shape Of The After Ouliers: ",dailyDataWithoutOutliers.shape)

Shape Of The Before Ouliers:  (10886, 15)
Shape Of The After Ouliers:  (10739, 15)


### 关联性分析
要理解因变量如何受到特征(数值)的影响，一个常见的方法是在它们之间建立一个相关矩阵。让我们在“计数”和“温度”、“温度”、“湿度”、“风速”之间绘制一个相关图。
* 温度和湿度特征分别与计数呈正相关和负相关，尽管两者之间的相关性不显著，但计数变量对“温度”和“湿度”的依赖性较小。
* 风速将不是真正有用的数值特征，从它与“计数”的相关值中可以看到它。
* 由于“temp”和“temp”之间有很强的相关性，所以“temp”是变量。在建模过程中，任何一个变量都必须被删除，因为它们将在数据中表现出多重共线性。
* “Casual”和“Registered”也没有被考虑在内，因为它们在性质上是泄漏变量，在建模过程中需要下降。
* 回归图是描述两个特征之间关系的有效方法之一。这里我们考虑“计数”与“温度”、“湿度”、“风速”。

In [18]:
corrMatt = dailyData[["temp","atemp","casual","registered","humidity","windspeed","count"]].corr()
mask = np.array(corrMatt)
mask[np.tril_indices_from(mask)] = False
fig,ax= plt.subplots()
fig.set_size_inches(20,10)
sn.heatmap(corrMatt, mask=mask,vmax=.8, square=True,annot=True)

<Figure size 1440x720 with 2 Axes>

In [19]:
fig,(ax1,ax2,ax3) = plt.subplots(ncols=3)
fig.set_size_inches(12, 5)
sn.regplot(x="temp", y="count", data=dailyData,ax=ax1)
sn.regplot(x="windspeed", y="count", data=dailyData,ax=ax2)
sn.regplot(x="humidity", y="count", data=dailyData,ax=ax3)

<Figure size 864x360 with 3 Axes>

### 可视化数据分布
如下图所示，“计数”变量向右倾斜。有正态分布是可取的，因为大多数机器学习技术都要求因变量为正态分布。一种可能的解决方案是删除异常数据点后对“计数”变量进行日志转换。在转换之后，数据看起来好多了，但仍然不理想地遵循正态分布。

In [20]:
fig,axes = plt.subplots(ncols=2,nrows=2)
fig.set_size_inches(12, 10)
sn.distplot(dailyData["count"],ax=axes[0][0])
stats.probplot(dailyData["count"], dist='norm', fit=True, plot=axes[0][1])
sn.distplot(np.log(dailyDataWithoutOutliers["count"]),ax=axes[1][0])
stats.probplot(np.log1p(dailyDataWithoutOutliers["count"]), dist='norm', fit=True, plot=axes[1][1])

((array([-3.82819677, -3.60401975, -3.48099008, ...,  3.48099008,
          3.60401975,  3.82819677]),
  array([0.69314718, 0.69314718, 0.69314718, ..., 6.5971457 , 6.59850903,
         6.5998705 ])),
 (1.3486990121229776, 4.562423868087808, 0.9581176780909617))

<Figure size 864x720 with 4 Axes>

### 可视化计数VS(月、季、小时、工作日、用户类型)
* 很明显，在夏季，人们倾向于租自行车，因为在那个季节骑自行车是非常有利的，所以六月、七月和八月对自行车的需求相对较高。
* 在平日，更多的人倾向于在早上7点到早上8点左右和下午5点到下午6点租自行车。正如我们前面提到的，这可归因于普通学校和办公室的上班族。
* 在“星期六”和“星期日”没有出现上述现象。更多的人倾向于在上午10点到下午4点之间租自行车。
* 繁忙时间约早上七时至上午八时及下午五时至六时，纯粹由注册用户贡献。

In [21]:
fig,(ax1,ax2,ax3,ax4)= plt.subplots(nrows=4)
fig.set_size_inches(12,20)
sortOrder = ["January","February","March","April","May","June","July","August","September","October","November","December"]
hueOrder = ["Sunday","Monday","Tuesday","Wednesday","Thursday","Friday","Saturday"]

monthAggregated = pd.DataFrame(dailyData.groupby("month")["count"].mean()).reset_index()
monthSorted = monthAggregated.sort_values(by="count",ascending=False)
sn.barplot(data=monthSorted,x="month",y="count",ax=ax1,order=sortOrder)
ax1.set(xlabel='Month', ylabel='Avearage Count',title="Average Count By Month")

hourAggregated = pd.DataFrame(dailyData.groupby(["hour","season"],sort=True)["count"].mean()).reset_index()
sn.pointplot(x=hourAggregated["hour"], y=hourAggregated["count"],hue=hourAggregated["season"], data=hourAggregated, join=True,ax=ax2)
ax2.set(xlabel='Hour Of The Day', ylabel='Users Count',title="Average Users Count By Hour Of The Day Across Season",label='big')

hourAggregated = pd.DataFrame(dailyData.groupby(["hour","weekday"],sort=True)["count"].mean()).reset_index()
sn.pointplot(x=hourAggregated["hour"], y=hourAggregated["count"],hue=hourAggregated["weekday"],hue_order=hueOrder, data=hourAggregated, join=True,ax=ax3)
ax3.set(xlabel='Hour Of The Day', ylabel='Users Count',title="Average Users Count By Hour Of The Day Across Weekdays",label='big')

hourTransformed = pd.melt(dailyData[["hour","casual","registered"]], id_vars=['hour'], value_vars=['casual', 'registered'])
hourAggregated = pd.DataFrame(hourTransformed.groupby(["hour","variable"],sort=True)["value"].mean()).reset_index()
sn.pointplot(x=hourAggregated["hour"], y=hourAggregated["value"],hue=hourAggregated["variable"],hue_order=["casual","registered"], data=hourAggregated, join=True,ax=ax4)
ax4.set(xlabel='Hour Of The Day', ylabel='Users Count',title="Average Users Count By Hour Of The Day Across User Type",label='big')

[Text(0, 0.5, 'Users Count'),
 Text(0.5, 0, 'Hour Of The Day'),
 Text(0.5, 1.0, 'Average Users Count By Hour Of The Day Across User Type'),
 None]

<Figure size 864x1440 with 4 Axes>

### 利用随机森林填充0的风速

In [22]:
dataTrain = pd.read_csv("/Users/ranmo/Desktop/机器学习项目集/华盛顿特区首都自行车租赁预测/data/train.csv")
dataTest = pd.read_csv("/Users/ranmo/Desktop/机器学习项目集/华盛顿特区首都自行车租赁预测/data/test.csv")

In [23]:
data = dataTrain.append(dataTest)
print(data.shape)
print(data.tail())
data.reset_index(inplace=True)
data.drop('index',inplace=True,axis=1)

(17379, 12)
       atemp  casual  count             datetime  holiday  humidity  \
6488  12.880     NaN    NaN  2012-12-31 19:00:00        0        60   
6489  12.880     NaN    NaN  2012-12-31 20:00:00        0        60   
6490  12.880     NaN    NaN  2012-12-31 21:00:00        0        60   
6491  13.635     NaN    NaN  2012-12-31 22:00:00        0        56   
6492  13.635     NaN    NaN  2012-12-31 23:00:00        0        65   

      registered  season   temp  weather  windspeed  workingday  
6488         NaN       1  10.66        2    11.0014           1  
6489         NaN       1  10.66        2    11.0014           1  
6490         NaN       1  10.66        1    11.0014           1  
6491         NaN       1  10.66        1     8.9981           1  
6492         NaN       1  10.66        1     8.9981           1  


/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [24]:
data["date"] = data.datetime.apply(lambda x : x.split()[0])
data["hour"] = data.datetime.apply(lambda x : x.split()[1].split(":")[0]).astype("int")
data["year"] = data.datetime.apply(lambda x : x.split()[0].split("-")[0])
data["weekday"] = data.date.apply(lambda dateString : datetime.strptime(dateString,"%Y-%m-%d").weekday())
data["month"] = data.date.apply(lambda dateString : datetime.strptime(dateString,"%Y-%m-%d").month)
print(data.head())

    atemp  casual  count             datetime  holiday  humidity  registered  \
0  14.395     3.0   16.0  2011-01-01 00:00:00        0        81        13.0   
1  13.635     8.0   40.0  2011-01-01 01:00:00        0        80        32.0   
2  13.635     5.0   32.0  2011-01-01 02:00:00        0        80        27.0   
3  14.395     3.0   13.0  2011-01-01 03:00:00        0        75        10.0   
4  14.395     0.0    1.0  2011-01-01 04:00:00        0        75         1.0   

   season  temp  weather  windspeed  workingday        date  hour  year  \
0       1  9.84        1        0.0           0  2011-01-01     0  2011   
1       1  9.02        1        0.0           0  2011-01-01     1  2011   
2       1  9.02        1        0.0           0  2011-01-01     2  2011   
3       1  9.84        1        0.0           0  2011-01-01     3  2011   
4       1  9.84        1        0.0           0  2011-01-01     4  2011   

   weekday  month  
0        5      1  
1        5      1  
2       

In [25]:
from sklearn.ensemble import RandomForestRegressor

dataWind0 = data[data["windspeed"]==0]
dataWindNot0 = data[data["windspeed"]!=0]
rfModel_wind = RandomForestRegressor()
windColumns = ["season","weather","humidity","month","temp","year","atemp"]
rfModel_wind.fit(dataWindNot0[windColumns], dataWindNot0["windspeed"])

wind0Values = rfModel_wind.predict(X= dataWind0[windColumns])
dataWind0["windspeed"] = wind0Values
data = dataWindNot0.append(dataWind0)
data.reset_index(inplace=True)
data.drop('index',inplace=True,axis=1)
print(data.head())

/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


    atemp  casual  count             datetime  holiday  humidity  registered  \
0  12.880     0.0    1.0  2011-01-01 05:00:00        0        75         1.0   
1  19.695    12.0   36.0  2011-01-01 10:00:00        0        76        24.0   
2  16.665    26.0   56.0  2011-01-01 11:00:00        0        81        30.0   
3  21.210    29.0   84.0  2011-01-01 12:00:00        0        77        55.0   
4  22.725    47.0   94.0  2011-01-01 13:00:00        0        72        47.0   

   season   temp  weather  windspeed  workingday        date  hour  year  \
0       1   9.84        2     6.0032           0  2011-01-01     5  2011   
1       1  15.58        1    16.9979           0  2011-01-01    10  2011   
2       1  14.76        1    19.0012           0  2011-01-01    11  2011   
3       1  17.22        1    19.0012           0  2011-01-01    12  2011   
4       1  18.86        2    19.9995           0  2011-01-01    13  2011   

   weekday  month  
0        5      1  
1        5      1  
2 

In [26]:
categoricalFeatureNames = ["season","holiday","workingday","weather","weekday","month","year","hour"]
numericalFeatureNames = ["temp","humidity","windspeed","atemp"]
dropFeatures = ['casual',"count","datetime","date","registered"]

In [27]:
for var in categoricalFeatureNames:
    data[var] = data[var].astype("category")

### 分割训练和测试数据

In [28]:
dataTrain = data[pd.notnull(data['count'])].sort_values(by=["datetime"])
dataTest = data[~pd.notnull(data['count'])].sort_values(by=["datetime"])
datetimecol = dataTest["datetime"]
yLabels = dataTrain["count"]
yLablesRegistered = dataTrain["registered"]
yLablesCasual = dataTrain["casual"]

### 删除无用变量

In [29]:
dataTrain  = dataTrain.drop(dropFeatures,axis=1)
dataTest  = dataTest.drop(dropFeatures,axis=1)

### 线性回归模型

In [30]:
def rmsle(y, y_,convertExp=True):
    if convertExp:
        y = np.exp(y),
        y_ = np.exp(y_)
    log1 = np.nan_to_num(np.array([np.log(v + 1) for v in y]))
    log2 = np.nan_to_num(np.array([np.log(v + 1) for v in y_]))
    calc = (log1 - log2) ** 2
    return np.sqrt(np.mean(calc))

In [31]:
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
import warnings
pd.options.mode.chained_assignment = None
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Initialize logistic regression model
lModel = LinearRegression()

# Train the model
yLabelsLog = np.log1p(yLabels)
lModel.fit(X = dataTrain,y = yLabelsLog)

# Make predictions
preds = lModel.predict(X= dataTrain)
print ("RMSLE Value For Linear Regression: ",rmsle(np.exp(yLabelsLog),np.exp(preds),False))

RMSLE Value For Linear Regression:  0.9779516580409092


In [32]:
predsTest = lModel.predict(X= dataTest)
fig,(ax1,ax2)= plt.subplots(ncols=2)
fig.set_size_inches(12,5)
sn.distplot(yLabels,ax=ax1,bins=50)
sn.distplot(np.exp(predsTest),ax=ax2,bins=50)

<Figure size 864x360 with 2 Axes>

### 集成模型-梯度升压

In [33]:
from sklearn.ensemble import GradientBoostingRegressor
gbm = GradientBoostingRegressor(n_estimators=4000,alpha=0.01); ### Test 0.41
yLabelsLog = np.log1p(yLabels)
gbm.fit(dataTrain,yLabelsLog)
preds = gbm.predict(X= dataTrain)
print ("RMSLE Value For Gradient Boost: ",rmsle(np.exp(yLabelsLog),np.exp(preds),False))

RMSLE Value For Gradient Boost:  0.19084027408592402


In [34]:
predsTest = gbm.predict(X= dataTest)
fig,(ax1,ax2)= plt.subplots(ncols=2)
fig.set_size_inches(12,5)
sn.distplot(yLabels,ax=ax1,bins=50)
sn.distplot(np.exp(predsTest),ax=ax2,bins=50)

<Figure size 864x360 with 2 Axes>

### 总结：
GradientBoostingRegressor算法比LR取得更好的预测效果！